In [8]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import functools
import operator
import re

In [9]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\user\Documents\Insper\2° semestre\CDados\Projetos\Projeto2_Cdados


In [12]:
dados = pd.read_excel('streaming_history_xlsx.xlsx')
dados

,Unnamed: 0,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Ouvi Dizer,0.641,0.614,7.0,-7.847,1.0,0.0532,0.0745,0.000000,0.093,0.727,81.011,audio_features,1qCLx3nX25NuhfPjM5E1qV,spotify:track:1qCLx3nX25NuhfPjM5E1qV,https://api.spotify.com/v1/tracks/1qCLx3nX25Nu...,https://api.spotify.com/v1/audio-analysis/1qCL...,210889.0,4.0
1,1,Pain 1993 (with Playboi Carti),0.825,0.373,5.0,-5.532,0.0,0.1080,0.1630,0.000000,0.368,0.120,77.271,audio_features,6Kj17Afjo1OKJYpf5VzCeo,spotify:track:6Kj17Afjo1OKJYpf5VzCeo,https://api.spotify.com/v1/tracks/6Kj17Afjo1OK...,https://api.spotify.com/v1/audio-analysis/6Kj1...,149078.0,4.0
2,2,Sentimento Bom,0.780,0.607,9.0,-6.440,1.0,0.0864,0.2060,0.000016,0.440,0.732,130.076,audio_features,0vyKzl0o1lem1sHg5wXgSD,spotify:track:0vyKzl0o1lem1sHg5wXgSD,https://api.spotify.com/v1/tracks/0vyKzl0o1lem...,https://api.spotify.com/v1/audio-analysis/0vyK...,241813.0,4.0
3,3,All Star,0.512,0.391,9.0,-10.400,1.0,0.0492,0.8890,0.000003,0.714,0.277,138.094,audio_features,3g5FrnRdbmDQyWNiDIprts,spotify:track:3g5FrnRdbmDQyWNiDIprts,https://api.spotify.com/v1/tracks/3g5FrnRdbmDQ...,https://api.spotify.com/v1/audio-analysis/3g5F...,233613.0,4.0
4,4,Contrato de RenovaÃ§Ã£o - Ao Vivo,0.577,0.752,9.0,-7.447,1.0,0.0654,0.7290,0.000000,0.929,0.844,90.213,audio_features,4C2eQoWJ1pkAFiZ0CDQ3Gt,spotify:track:4C2eQoWJ1pkAFiZ0CDQ3Gt,https://api.spotify.com/v1/tracks/4C2eQoWJ1pkA...,https://api.spotify.com/v1/audio-analysis/4C2e...,226951.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1125,1126,Sozinho - Ao Vivo,0.481,0.284,11.0,-14.762,0.0,0.1860,0.6340,0.000168,0.910,0.528,83.979,audio_features,02a8cGumnKuEPgoCzmalJp,spotify:track:02a8cGumnKuEPgoCzmalJp,https://api.spotify.com/v1/tracks/02a8cGumnKuE...,https://api.spotify.com/v1/audio-analysis/02a8...,190533.0,3.0
1126,1127,Pisando DescalÃ§o,0.767,0.508,4.0,-8.599,0.0,0.0894,0.4200,0.000000,0.190,0.557,79.023,audio_features,65XFXZfU748XLkaNJYCTHV,spotify:track:65XFXZfU748XLkaNJYCTHV,https://api.spotify.com/v1/tracks/65XFXZfU748X...,https://api.spotify.com/v1/audio-analysis/65XF...,189325.0,4.0
1127,1128,Poesia AcÃºstica - Paris,0.696,0.748,1.0,-4.747,0.0,0.0822,0.4610,0.000000,0.172,0.494,87.981,audio_features,5JKaHAt4atuNoLqpyDgbyK,spotify:track:5JKaHAt4atuNoLqpyDgbyK,https://api.spotify.com/v1/tracks/5JKaHAt4atuN...,https://api.spotify.com/v1/audio-analysis/5JKa...,420000.0,4.0
1128,1129,Libera Ela - Ao Vivo,0.769,0.649,0.0,-4.864,1.0,0.0714,0.3840,0.000000,0.949,0.834,77.487,audio_features,0SJtvFn4vqgJPscSbEPaql,spotify:track:0SJtvFn4vqgJPscSbEPaql,https://api.spotify.com/v1/tracks/0SJtvFn4vqgJ...,https://api.spotify.com/v1/audio-analysis/0SJt...,168080.0,4.0
